# Evaluating models on the dataset

- Caricare i video
- Preprocessing dei video (resize e normalizzazione)
- Split train test
- Estrazione features con CNN
- passaggio features a LSTM per classificazione

In [4]:
import tensorflow as tf

2.8.0
